# tourapi

In [9]:
## restaurant

In [2]:
import pandas as pd
tourapi = pd.read_excel('/home/hwang/projects/yanolja/data/dataset/tourAPI/restaurant.xls')

In [3]:
tourapi['source'] = 'ToruAPI'

In [4]:
tourapi['category'] = "restaurant"

In [5]:
tourapi.head(2)

,명칭,우편번호,관리자,전화번호,주소,위도,경도,개요,문의 및 안내,규모,...,취급메뉴,할인정보,금연/흡연,신용카드정보,포장가능,예약안내,상세정보,인허가번호,source,category
0,파크 하얏트 서울 코너스톤,6174,파크 하얏트 서울,02-2016-1220~1221,서울특별시 강남구 테헤란로 6062층,37.508715,127.064265,"파크 하얏트 서울의 코너스톤(Cornerstone, Cucina Italiana)은...",02-2016-1220,NaN,...,NaN,NaN,모두 금연석,가능,가능(일부 메뉴),"전화 예약 가능 (02-2016-1220), 온라인 예약 (http://seoul....","부대시설:남녀 개별 화장실 / 업소내 위치\n부대시설:별실 4개, 바\n",20050105616,ToruAPI,restaurant
1,장마담집,13131,장마담집,031-745-5700,경기도 성남시 수정구 수정로 460-3(단대동),37.461603,127.155825,장마담집은 성남 남한산성 입구에 위치한 곳으로 3대째 운영중인 전통있는 백숙 전문 ...,031-745-5700,NaN,...,닭볶음탕 / 토종닭백숙 / 오리탕 / 해물파전 외,NaN,모두 금연석,가능,가능,전화 예약 문의 (031-745-5700),화장실:남녀개별 / 업소 내부 위치\n,NaN,ToruAPI,restaurant


In [6]:
tourapi.columns

Index(['명칭', '우편번호', '관리자', '전화번호', '주소', '위도', '경도', '개요', '문의 및 안내', '규모',
       '좌석수', '주차 시설', '어린이놀이방', '개업일', '영업시간', '쉬는날', '대표메뉴', '취급메뉴', '할인정보',
       '금연/흡연', '신용카드정보', '포장가능', '예약안내', '상세정보', '인허가번호', 'source',
       'category'],
      dtype='object')

In [7]:
tourapi.rename(columns= {'명칭': 'place_name', #
                                   '주소':'address', #
                                   '위도': 'latitude', #
                                   '경도': 'longitude', #
                                   '주차 시설': 'parking', #
                                   '개요': 'description', #
                                   '우편번호': 'zip_code', #
                                   '관리자': 'manager', #
                                   '전화번호': 'phone', # 
                                   '문의 및 안내': 'guidance', #
                                   '규모': 'scale', #
                                   '좌석수': 'seats',
                                   '어린이놀이방': 'children_playroom',
                                   '개업일': 'opening_date',
                                   '영업시간': 'available_time',
                                   '대표메뉴': 'main_menu',
                                   '취급메뉴': 'menus',
                                   '신용카드정보': 'credit_card',
                                   '포장가능': 'packable',
                                   '예약안내': 'reservation', 
                                   '인허가번호': 'license_num',
                                   '쉬는날': 'day_off', # 
                                   '할인정보': 'discount_info', #
                                   '상세정보': 'details'}, inplace=True) # 

tourapi = tourapi[['source', 'category', 'place_name', 'address', 'latitude', 'longitude', 'available_time', 'parking', 'description', 'zip_code', 
                   'manager', 'phone', 'guidance', 'scale', 'seats', 'children_playroom', 'opening_date', 'main_menu', 'menus',
                   'packable',  'reservation', 'license_num', 'day_off', 'discount_info', 'credit_card', 'details']]

In [8]:
tourapi.isnull().sum() / len(tourapi)

source               0.000000
category             0.000000
place_name           0.000000
address              0.000062
latitude             0.000000
longitude            0.000000
available_time       0.004141
parking              0.135591
description          0.000618
zip_code             0.000989
manager              0.719609
phone                0.689698
guidance             0.002966
scale                1.000000
seats                0.970397
children_playroom    0.002534
opening_date         0.962240
main_menu            0.008714
menus                0.018602
packable             0.460169
reservation          0.517026
license_num          0.040356
day_off              0.066992
discount_info        0.998146
credit_card          0.184970
details              0.405970
dtype: float64

In [9]:
tourapi.isnull().sum()

source                   0
category                 0
place_name               0
address                  1
latitude                 0
longitude                0
available_time          67
parking               2194
description             10
zip_code                16
manager              11644
phone                11160
guidance                48
scale                16181
seats                15702
children_playroom       41
opening_date         15570
main_menu              141
menus                  301
packable              7446
reservation           8366
license_num            653
day_off               1084
discount_info        16151
credit_card           2993
details               6569
dtype: int64

In [24]:
import openai
import pandas as pd
import re
import json
import os
from tqdm import tqdm
from dotenv import load_dotenv
import time

# Load environment variables
if not load_dotenv(dotenv_path="/home/hwang/projects/yanolja/src/.env"):
    print(".env 파일을 로드하지 못했습니다. 경로를 확인하세요.")
openai.api_key = os.getenv("OPENAI_API_KEY")
if openai.api_key is None:
    raise ValueError("OpenAI API 키가 설정되지 않았습니다. .env 파일을 확인하세요.")

# Load DataFrame
df = tourapi[15200:]

# Preprocess text
def preprocess_text(text):
    if pd.isna(text) or text.strip() == "":
        return "정보 없음"
    text = re.sub(r"<[^>]+>", " ", text)  # Remove HTML tags
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra whitespace
    return text

df["day_off"] = df["day_off"].apply(preprocess_text)
df["available_time"] = df["available_time"].apply(preprocess_text)

# Transform function with error handling and retry logic
def transform_to_standard_format(day_off, available_time, max_retries=3, retry_delay=5):
    day_off = preprocess_text(str(day_off))
    available_time = preprocess_text(str(available_time))
    
    prompt = f"""
    **입력 데이터**:
    - 영업시간(available_time): {available_time}
    - 휴무일(day_off): {day_off}

    입력 데이터에 대해서 이해한 바를 창조하지 말고 사실에 기반하여 서술하세요.
    """
    
    retries = 0
    while retries < max_retries:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-0125",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0,
                max_tokens=300
            )
            return response['choices'][0]['message']['content']
        except openai.error.OpenAIError as e:
            print(f"OpenAI API 에러 발생: {e}. 재시도 중 ({retries + 1}/{max_retries})...")
            retries += 1
            time.sleep(retry_delay)
        except Exception as e:
            print(f"예상치 못한 에러 발생: {e}")
            break
    
    # If all retries fail, return a default error message
    return "API 요청 실패: 데이터를 처리하지 못했습니다."

# Process rows
results = []
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
    try:
        transformed = transform_to_standard_format(row["day_off"], row["available_time"])
        results.append({"row_index": index, "transformed_data": transformed})
    except Exception as e:
        print(f"Row {index} 처리 중 에러 발생: {e}")
        results.append({"row_index": index, "transformed_data": "처리 실패"})

# Save results to a JSON file
output_file = "transformed_tourapi_restaurant_available_time_results_v1_3.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"변환된 결과가 {output_file} 파일에 저장되었습니다.")


/tmp/ipykernel_65955/2605001663.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["day_off"] = df["day_off"].apply(preprocess_text)
/tmp/ipykernel_65955/2605001663.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["available_time"] = df["available_time"].apply(preprocess_text)
Processing rows:   0%|          | 0/981 [00:00<?, ?it/s]

Processing rows: 100%|██████████| 981/981 [24:39<00:00,  1.51s/it]

변환된 결과가 transformed_tourapi_restaurant_available_time_results_v1_3.json 파일에 저장되었습니다.


In [25]:
import json
import openai

import os
from dotenv import load_dotenv

from tqdm import tqdm

load_dotenv(dotenv_path="/home/hwang/projects/yanolja/src/.env")
openai.api_key = os.getenv("OPENAI_API_KEY")

input_file = "/home/hwang/projects/yanolja/src/collecting_data/data_analysis/transformed_tourapi_restaurant_available_time_results_v1_3.json"
output_file = "transformed_tourapi_restaurant_available_time_results_v2_3.json"

def transform_data_with_gpt(input_text):
    system_prompt = (
        "You are an assistant that transforms text about business hours and holidays into a structured JSON format. "
        "The JSON must strictly adhere to the following rules and reflect only the information explicitly provided in the input text. "
        "If the input specifies different 'entry closing time' (e.g., last entry allowed) and 'closing time' (e.g., business hours end), these must be clearly separated:\n\n"
        "Rules for transformation:\n"
        "1. The 'hours' key should only include the official business hours for each day (Monday to Sunday).\n"
        "   - If no business hours are provided for a specific day, leave that day as an empty list (e.g., \"Monday\": []).\n"
        "   - 'Closing time' refers to the time when the business officially stops operating and should be used in the 'hours' key.\n\n"
        "2. The 'special_conditions' key should include:\n"
        "   - 'entry closing time' (last entry allowed) as a separate condition with 'type: \"entry_closing_time\"' if explicitly stated.\n"
        "   - Any holidays, seasonal hours, or other exceptions.\n"
        "   - Each entry must have a 'type' and 'details'.\n\n"
        "3. If conflicting or ambiguous information is present in the input, separate the conflicting details into individual entries in 'special_conditions'.\n\n"
        "4. Do not infer or assume any information. Only use the data explicitly provided in the input text.\n"
        "   - If any information is missing or incomplete, leave the corresponding field empty (e.g., `[]`, `{}`).\n"
        "   - Only add 'details: \"Information not provided\"' if explicitly mentioned in the input text."
    )

    user_prompt = (
        f"Transform the following text into the required JSON format. "
        "Ensure that the output strictly adheres to the rules provided and includes only the information explicitly stated in the input text. "
        "If entry closing time (last entry allowed) and closing time (business end) are mentioned, clearly separate them. "
        "Add entry closing time as a 'special_condition' with 'type: \"entry_closing_time\"'. "
        "If information is missing, ambiguous, or incomplete, leave fields empty (e.g., `[]`, `{}`) and do not make assumptions:\n\n"
        f"Text: {input_text}\n\n"
        "Output example:\n"
        "{\n"
        '  "hours": {\n'
        '    "Monday": [],\n'
        '    "Tuesday": [{"start": "10:00", "end": "18:00"}],\n'
        '    "Wednesday": [{"start": "10:00", "end": "18:00"}],\n'
        '    "Thursday": [{"start": "10:00", "end": "18:00"}],\n'
        '    "Friday": [{"start": "10:00", "end": "18:00"}],\n'
        '    "Saturday": [{"start": "10:00", "end": "18:00"}],\n'
        '    "Sunday": [{"start": "10:00", "end": "18:00"}]\n'
        "  },\n"
        '  "special_conditions": [\n'
        '    {"type": "holiday", "details": "Closed on public holidays"},\n'
        '    {"type": "holiday", "details": "Closed on New Year\'s Day"},\n'
        '    {"type": "holiday", "details": "Closed on Lunar New Year and Chuseok"},\n'
        '    {"type": "entry_closing_time", "details": "Entry closes at 17:30 every day"}\n'
        "  ]\n"
        "}"
    )

    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0
    )
    
    return response["choices"][0]["message"]["content"]

with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

transformed_data = []

for item in tqdm(data):
    try:
        input_text = item.get("transformed_data", "")
        transformed_json = transform_data_with_gpt(input_text)  
        transformed_data.append({
            "row_index": item["row_index"],
            "transformed_data": json.loads(transformed_json)  
        })
    except Exception as e:
        print(f"Error processing row_index {item['row_index']}: {e}")

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(transformed_data, f, ensure_ascii=False, indent=4)

print(f"Transformed data saved to {output_file}")


 47%|████▋     | 459/981 [25:44<26:18:53, 181.48s/it]

Error processing row_index 15658: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)


100%|██████████| 981/981 [43:51<00:00,  2.68s/it]    

Transformed data saved to transformed_tourapi_restaurant_available_time_results_v2_3.json


In [10]:
import json
import pandas as pd

file_path1 = "/home/hwang/projects/yanolja/src/collecting_data/data_analysis/transformed_tourapi_restaurant_available_time_results_v2_1.json"
file_path2 = "/home/hwang/projects/yanolja/src/collecting_data/data_analysis/transformed_tourapi_restaurant_available_time_results_v2_2.json"
file_path3 = "/home/hwang/projects/yanolja/src/collecting_data/data_analysis/transformed_tourapi_restaurant_available_time_results_v2_3.json"


with open(file_path1, 'r', encoding='utf-8') as f1:
    at1 = json.load(f1)

with open(file_path2, 'r', encoding='utf-8') as f2:
    at2 = json.load(f2)

with open(file_path2, 'r', encoding='utf-8') as f3:
    at3 = json.load(f3)

available_time1 = pd.DataFrame(at1)
available_time2 = pd.DataFrame(at2)
available_time3 = pd.DataFrame(at3)

tmp = pd.concat([available_time1, available_time2, available_time3], axis=0)

tourapi = pd.concat([tourapi.reset_index(drop=True), tmp.reset_index(drop=True)], axis=1)
tourapi = tourapi.loc[:, ~tourapi.columns.duplicated()] 

tourapi.rename(columns={'transformed_data': 'available_time_v2'}, inplace=True)

In [11]:
tourapi = tourapi[~tourapi['place_name'].isna()]

In [12]:
tourapi[((tourapi['latitude'] <= 33) | (tourapi['latitude'] >= 43)) & ((tourapi['longitude'] <= 124) | (tourapi['longitude'] >= 132))]

,source,category,place_name,address,latitude,longitude,available_time,parking,description,zip_code,...,menus,packable,reservation,license_num,day_off,discount_info,credit_card,details,row_index,available_time_v2
11311,ToruAPI,restaurant,정원오리촌,부산광역시 강서구 명지오션시티10로 12731-10,9.999823,9.999729,11:40~22:00,있음/주차요금 무료,명지동 행복마을 안에 있는 정원 오리촌은 넓은 매장에 빈자리가 없을 만큼 유명한 맛...,46763,...,"오리탕, 오리양념구이, 오리훈제 등",NaN,NaN,20190149401,연중무휴,NaN,가능,부대시설:남녀구분\n,11326,"{'hours': {'Monday': [{'start': '10:00', 'end'..."


In [13]:
from dotenv import load_dotenv
import os
import numpy as np

import importlib
import analysis_utils as analysis_utils 
importlib.reload(analysis_utils)

from analysis_utils import get_geocode

load_dotenv()

client_id = os.getenv("naver_geocoding_client_id")
client_secret = os.getenv("naver_geocoding_client_secret")

# query = "서울특별시 강남구 가로수길 69 엘큐브elcube"

# geocode = get_geocode(query=query, client_id=client_id, client_secret=client_secret)
# geocode

query = tourapi.iloc[11311]['address']
geocode = get_geocode(query=query, client_id=client_id, client_secret=client_secret)

tourapi.loc[11311, 'latitude'] = np.float64(geocode['latitude'])
tourapi.loc[11311, 'longitude'] = np.float64(geocode['longitude'])

TypeError: 'float' object is not subscriptable

In [14]:
tourapi = tourapi.drop(index=11311)
tourapi = tourapi.reset_index(drop=True)

In [15]:
tourapi[((tourapi['latitude'] <= 33) | (tourapi['latitude'] >= 43)) & ((tourapi['longitude'] <= 124) | (tourapi['longitude'] >= 132))]

,source,category,place_name,address,latitude,longitude,available_time,parking,description,zip_code,...,menus,packable,reservation,license_num,day_off,discount_info,credit_card,details,row_index,available_time_v2


In [16]:
tourapi[tourapi['address'].isnull()]

,source,category,place_name,address,latitude,longitude,available_time,parking,description,zip_code,...,menus,packable,reservation,license_num,day_off,discount_info,credit_card,details,row_index,available_time_v2
14497,ToruAPI,restaurant,그랑깨,NaN,35.405881,127.378238,11:00 - 18:20,NaN,그랑깨 돈까스는 남원 현지인 맛집으로 유명하다. 대표메뉴인 냉모밀과 안심 치즈 매콤...,NaN,...,돈까스 / 냉모밀,NaN,NaN,NaN,매주 월요일,NaN,NaN,NaN,14520,"{'hours': {'Monday': [{'start': '11:00', 'end'..."


In [17]:
from dotenv import load_dotenv
import os
import numpy as np

import importlib
import analysis_utils as analysis_utils
importlib.reload(analysis_utils)

from analysis_utils import get_reverse_geocode

load_dotenv()

client_id = os.getenv("naver_geocoding_client_id")
client_secret = os.getenv("naver_geocoding_client_secret")

# latitude = 37.472404
# longitude = 126.634438

# reverse_geocode = get_reverse_geocode(latitude=latitude, longitude=longitude, client_id=client_id, client_secret=client_secret)
# reverse_geocode

latitude, longitude = tourapi.loc[14497, ['latitude', 'longitude']]
reverse_geocode = get_reverse_geocode(latitude=latitude, longitude=longitude, client_id=client_id, client_secret=client_secret)

tourapi.loc[14497, 'address'] = reverse_geocode['address'].strip()

In [18]:
tourapi[tourapi['address'].isnull()]

,source,category,place_name,address,latitude,longitude,available_time,parking,description,zip_code,...,menus,packable,reservation,license_num,day_off,discount_info,credit_card,details,row_index,available_time_v2


In [19]:
tourapi[tourapi['description'].isnull()]

,source,category,place_name,address,latitude,longitude,available_time,parking,description,zip_code,...,menus,packable,reservation,license_num,day_off,discount_info,credit_card,details,row_index,available_time_v2
7004,ToruAPI,restaurant,서화,서울특별시 중구 서소문로 109 (서소문동),37.563052,126.973235,NaN,NaN,NaN,4515,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7013,"{'hours': {'Monday': [], 'Tuesday': [{'start':..."
9006,ToruAPI,restaurant,금정산성 유가네누룩,부산광역시 금정구 산성로 453 (금성동),35.250412,129.056428,"평일 10:00~15:00, 주말 가능(5명 이상 시, 2주 전 예약 원칙)",있음(무료),NaN,46237,...,NaN,NaN,NaN,NaN,주말,NaN,NaN,화장실:있음(남/여 구분)\n,9016,"{'hours': {'Monday': [{'start': '12:00', 'end'..."
10084,ToruAPI,restaurant,오대산 달빛미소,강원특별자치도 평창군 진부면 오대산로 152,37.716711,128.601905,08:30 ~ 19:00(동절기 불규칙),있음,NaN,25318,...,"산채정식, 한식",NaN,NaN,20170397137,상시 변동,NaN,가능,NaN,10097,"{'hours': {'Monday': [{'start': '11:00', 'end'..."
10475,ToruAPI,restaurant,하누,강원특별자치도 평창군 대화면 대화3길 11나동,37.500997,128.453362,매일 10:00 ~ 21:30,있음 10여대가능,NaN,25358,...,"소고기구이,갈비탕,도가니탕,육개장",NaN,NaN,20130397105,NaN,NaN,없음,NaN,10489,"{'hours': {'Monday': [{'start': '11:00', 'end'..."
11723,ToruAPI,restaurant,담돌국수,제주특별자치도 제주시 삼성로 67,33.508141,126.532367,전화문의,NaN,NaN,63268,...,고기국수 / 닭칼국수 / 빨간고기국수 / 비빔국수 외,NaN,NaN,19970627682,전화문의,NaN,없음,NaN,11742,"{'hours': {'Monday': [{'start': '11:00', 'end'..."
11787,ToruAPI,restaurant,오아시스 청담점,서울특별시 강남구 도산대로55길 20영빌딩 1층,37.524830,127.040619,매주 09:00 ~ 18:00(17:00 라스트오더),있음(대리 주차) / 대리 주차 유료,NaN,6014,...,"타이 누들 샐러드 해물, 애플햄치즈프렌치토스트, 치킨 아보카도 샐러드, 런치 파스타...",가능,불가능,20090105840,연중무휴,NaN,가능,부대시설:무선 인터넷\n화장실:남녀 구분\n,11806,"{'hours': {'Monday': [], 'Tuesday': [{'start':..."
12674,ToruAPI,restaurant,도성,서울특별시 종로구 자하문로7길 15,37.578503,126.970794,11:00-21:00,가능,NaN,3040,...,짜장면,NaN,가능,NaN,명절,NaN,없음,NaN,12694,"{'hours': {'Monday': [], 'Tuesday': [{'start':..."
14088,ToruAPI,restaurant,울릉도 All in one! 로컬의 이야기 가득 담긴 '저동커피',경상북도 울릉군 울릉읍 울릉순환로 164-2,37.496317,130.909629,NaN,NaN,NaN,40218,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14111,"{'hours': {'Monday': [], 'Tuesday': [{'start':..."
16035,ToruAPI,restaurant,오우가,서울특별시 중구 명동8가길 42지하1층,37.561747,126.987412,10:00-24:00,NaN,NaN,4537,...,대나무쌈밥정식 / 생갈비살정식,NaN,NaN,NaN,명절연휴 휴무,NaN,없음,NaN,8862,"{'hours': {'Monday': [], 'Tuesday': [{'start':..."
16063,ToruAPI,restaurant,광명식당,"전라남도 여수시 학동2길 (7-4, 제일시장)",34.764236,127.664679,방문전 전화문의 요망,NaN,NaN,59677,...,쭈꾸미삼합 / 삼겹살 / 장어구이 / 서대회 등,NaN,NaN,19890506114,NaN,NaN,NaN,NaN,8890,"{'hours': {'Monday': [], 'Tuesday': [{'start':..."


In [20]:
tourapi[tourapi['description'].isnull()]['place_name'].tolist()

['서화',
 '금정산성 유가네누룩',
 '오대산 달빛미소',
 '하누',
 '담돌국수',
 '오아시스 청담점',
 '도성',
 "울릉도 All in one! 로컬의 이야기 가득 담긴 '저동커피'",
 '오우가',
 '광명식당']

In [21]:
tourapi.loc[tourapi['place_name']=='서화', 'description'] = np.NaN
tourapi.loc[tourapi['place_name']=='금정산성 유가네누룩', 'description'] = "금정산성유가네누룩은 500년 동안 이어온 유가네 조상들의 경험과 지혜가 고스란히 보존되어 문화제적 가치가 높은 누룩입니다. 전통 누룩을 사용해 발효시켜 만들기 때문에 일반 시중 막걸리보다 산미가 강한 것이 특징입니다."
tourapi.loc[tourapi['place_name']=='오대산 달빛미소', 'description'] = "오대산 달빛미소는 오대산 국립공원 인근에 위치한 식당으로, 지역의 신선한 제철 식재료를 사용해 건강하고 맛있는 한식을 제공합니다. "
tourapi.loc[tourapi['place_name']=='하누', 'description'] = np.NaN
tourapi.loc[tourapi['place_name']=='담돌국수', 'description'] = "담돌국수는 제주시내에 위치한 고기국수 맛집입니다. 대표 메뉴는 고기국수와 돔베고기입니다. "
tourapi.loc[tourapi['place_name']=='오아시스 청담점', 'description'] = "‘오아시스’는 태국, 영국, 호주 등 여러 나라의 브런치를 한 자리에서 맛볼 수 있는 곳입니다. 직접 만든 빵과 소스를 이용하여 건강한 브런치를 선보입니다. 대표 메뉴는 빵에 햄과 수란을 쌓은 뒤 홀랜다이즈 소스로 마무리한 ‘에그 베네딕트 햄’입니다. 짭조름한 햄과 고소한 수란을 소스가 상큼하게 감싸며 조화롭게 어우러집니다."
tourapi.loc[tourapi['place_name']=='도성', 'description'] = "깔끔한 인테리어와 북적거리는 사람들로 눈길을 끄는 곳! 위치는 우리은행 근처라 찾기 어렵지 않다. 입구로 들어서면 복도 양쪽으로 밀폐된 룸이 준비되어 있다. 상견례나 조용한 분위기에서 식사를 원하는 사람들이 예약을 하고 이용하는 곳이다. 배달도 가능해서 동네 주민들은 자주 이용하는 곳이기도 하다."
tourapi.loc[tourapi['place_name']=="울릉도 All in one! 로컬의 이야기 가득 담긴 '저동커피'", 'description'] = np.NaN
tourapi.loc[tourapi['place_name']=='오우가', 'description'] = "이 곳은 대나무 쌈밥이 인기인 음식점입니다. 일본인 관광객들이 많은 명동에 위치하여 일본인들도 많이 찾는 곳이라고 합니다. 그만큼 영어나 일어를 필수로 할 줄 아는 종업원분들이 계시답니다. 또한 이 곳에서는 가게에서 직접 만든 쌈장맛이 일품이라고 하는데, 쌈장에는 여러가지 씨앗이 들어가있으며 간이짜지 않아 좋다고 합니다. 건강한 밥 한끼를 하고 싶으시다면 이 곳을 찾아 오는 것도 좋을 듯 싶습니다."
tourapi.loc[tourapi['place_name']=='광명식당', 'description'] = "전남 여수에 있는 광명식당은 쭈꾸미와 삼합을 맛볼 수 있는 현지인 맛집입니다. "



In [22]:
import importlib
import analysis_utils as analysis_utils
importlib.reload(analysis_utils)

from analysis_utils import get_wikipedia_summary

tourist_spot = "잠월미술관"
summary = get_wikipedia_summary(tourist_spot)


In [23]:
tourapi[tourapi['parking'].isnull()]

,source,category,place_name,address,latitude,longitude,available_time,parking,description,zip_code,...,menus,packable,reservation,license_num,day_off,discount_info,credit_card,details,row_index,available_time_v2
31,ToruAPI,restaurant,키다리아저씨,서울특별시 마포구 성미산로 22,37.558426,126.910722,08:30~21:30,NaN,20년 이상 경력의 제빵 경력으로 동네 맛집으로 입소문 난 곳이다. \n매달 마지막...,3996,...,빵 / 케이크 / 커피 / 마카롱 / 스콘 / 앙버터 / 코코넛쿠키 외,가능,NaN,20130069472,"추석, 설날 포함 앞뒤 3일 휴무",NaN,가능,NaN,31,"{'hours': {'Monday': [], 'Tuesday': [{'start':..."
32,ToruAPI,restaurant,썬시티 레스토랑,경상북도 경주시 양남면 동남로 1021,35.684339,129.369322,일-목 08:00 ~ 10:00<br>금-토 17:00 ~ 21:00,NaN,동해 바다와 동대산을 배경으로 해발 500m 고지에서 식사를 즐길 수 있는 곳으로 ...,38221,...,골뱅이무침 / 쭈꾸미&삼겹살 두루치기 외,NaN,전화 예약 문의 (054-740-0566),20090538006,연중무휴(전화문의),NaN,가능,"화장실:남녀개별\n\n장애인 편의시설:주출입구에 경사로 설치, 장애인전용 주차구역,...",32,"{'hours': {'Monday': [{'start': '08:00', 'end'..."
42,ToruAPI,restaurant,두머리부엌,경기도 양평군 양서면 용담5길 23A동 101,37.546088,127.326984,11:30~20:00 (브레이크타임 15:00~17:00)<br>※ 주말 브레이크타...,NaN,2014년 문을 연 두머리부엌은 협동조합으로 모인 양수리 마을 사람들의 특별한 공간...,12584,...,시시백반 / 국산콩 두부구이 / 무농약 감자전 / 제육볶음 외,NaN,NaN,20140378181,NaN,NaN,가능,NaN,42,"{'hours': {'Monday': [{'start': '11:30', 'end'..."
49,ToruAPI,restaurant,[백년가게]부산복집,서울특별시 중구 수표로2길 25,37.562302,126.991914,10:00~21:30 (라스트오더 21:00),NaN,부산복집은＂백년가게＂로 선정된 바 있는 복어요리 전문점이다. 복어 요리가 흔하지 않...,4554,...,복매운탕 / 복찜 / 복수육 등,가능(2인분 이상),"전화, 온라인 예약가능",19880029160,명절,NaN,가능,화장실:업소내 위치\n,49,"{'hours': {'Monday': [], 'Tuesday': [{'start':..."
55,ToruAPI,restaurant,[백년가게]마방(마일드치킨),충청남도 서산시 번화1로 41-1,36.782775,126.453957,16:00~23:30,NaN,[중소벤처기업부 2019년도 '백년가게'로 선정]<br /><br />마방(마일드치...,31981,...,치킨(후라이드·양념) / 골뱅이무침 외,가능,NaN,19820463040,매월 첫째주 일요일,NaN,가능,부대시설:남녀개별\n,55,"{'hours': {'Monday': [{'start': '16:00', 'end'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16173,ToruAPI,restaurant,광장동 가온,서울특별시 중구 명동길 74 명동성당,37.563791,126.986637,11:00 ~ 20:30,NaN,“내 가족 안심 먹거리”라는 정신하에 2008년 설립된 ‘광장동가온’은 대종손집안의...,4537,...,가온곰국시 / 떡만두국 / 바싹불고기 외,NaN,NaN,20170029291,연중무휴,NaN,NaN,NaN,9000,"{'hours': {'Monday': [{'start': '10:00', 'end'..."
16174,ToruAPI,restaurant,동남식당,경상북도 경주시 해변공원길 61,35.674256,129.463686,11:00 ~ 15:00 (라스트 오더 오후2시 30분),NaN,경주 양남 하서리의 한적한 바닷가 마을에 위치한 회국수 전문점인 동남식당이다. 이 ...,38220,...,"횟밥, 회국수, 잔치국수",NaN,NaN,NaN,월요일,NaN,NaN,NaN,9001,"{'hours': {'Monday': [{'start': '11:00', 'end'..."
16175,ToruAPI,restaurant,퍼플스왈로우,인천광역시 중구 영종진로66번길 15,37.499305,126.576207,10:00 ~ 20:00 (라스트오더 19:20),NaN,"인천 영종도에 위치한 분위기 있는 카페인 퍼플스왈로우, 영정도의 옛 지명인 자주 빛...",22393,...,"스왈로우 커피, 딸기 콩포트 크림 라떼",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9002,"{'hours': {'Monday': [{'start': '11:00', 'end'..."
16176,ToruAPI,restaurant,새봄떡국국수,서울특별시 중구 새문안로 24,37.567451,126.968602,09:00 ~ 21:30,NaN,서울에 위치한 떡국과 국수를 전문으로 하고 있는 음식점이다. 가게 외부와 내부는 평...,4516,...,활전복떡국 / 매생이 굴 떡국 / 굴 버섯 떡국 / 소고기 떡국 외,가능,NaN,NaN,NaN,NaN,가능,NaN,9003,"{'hours': {'Monday': [], 'Tuesday': [{'start':..."


In [24]:
import pandas as pd

parking = pd.read_excel("/home/hwang/projects/yanolja/data/dataset/etc/전국주차장정보표준데이터-20241210.xls")
parking.columns = parking.iloc[0]
parking = parking[1:].reset_index(drop=True)
parking['위도'] = pd.to_numeric(parking['위도'], errors="coerce")
parking['경도'] = pd.to_numeric(parking['경도'], errors="coerce")
parking = parking.dropna(subset=['위도', '경도'])

In [25]:
parking.head(2)

,주차장관리번호,주차장명,주차장구분,주차장유형,소재지도로명주소,소재지지번주소,주차구획수,급지구분,부제시행구분,운영요일,...,결제방법,특기사항,관리기관명,전화번호,위도,경도,장애인전용주차구역보유여부,데이터기준일자,제공기관코드,제공기관명
0,204-2-000023,아래숯골,공영,노외,경기도 성남시 수정구 성남대로 1259,NaN,150,1,요일제,평일+토요일+공휴일,...,신용카드,경차+장애인 차량 50프로 할인 등,성남도시개발공사 노외주차처,NaN,37.442288,127.126662,Y,2024-05-31,3780000,경기도 성남시
1,204-2-000024,태평4동,공영,노외,경기도 성남시 수정구 태평로 109,NaN,89,1,요일제,평일+토요일+공휴일,...,신용카드,경차+장애인 차량 50프로 할인 등,성남도시개발공사 노외주차처,NaN,37.451234,127.138580,Y,2024-05-31,3780000,경기도 성남시


In [26]:
import importlib
import analysis_utils as analysis_utils
importlib.reload(analysis_utils)

from analysis_utils import find_nearest_parking

nearest_parking = find_nearest_parking(poi_lat=37.610198, poi_lon=127.040989, parking_data=parking)

In [27]:
nearest_parking

'동방고개 공영 주차장'

In [28]:
tourapi.columns

Index(['source', 'category', 'place_name', 'address', 'latitude', 'longitude',
       'available_time', 'parking', 'description', 'zip_code', 'manager',
       'phone', 'guidance', 'scale', 'seats', 'children_playroom',
       'opening_date', 'main_menu', 'menus', 'packable', 'reservation',
       'license_num', 'day_off', 'discount_info', 'credit_card', 'details',
       'row_index', 'available_time_v2'],
      dtype='object')

In [29]:
import pandas as pd
import numpy as np

def combine_columns(row):
    additional_info = []
    # children playroom
    if pd.notna(row['children_playroom']):
        additional_info.append(f"어린이놀이터: {row['children_playroom']}")

    # main menu 추가
    if pd.notna(row['main_menu']):
        additional_info.append(f"메인메뉴: {row['main_menu']}")
    
    # menus 추가
    if pd.notna(row['menus']):
        additional_info.append(f"메뉴: {row['menus']}")
    
    # reservation 추가
    if pd.notna(row['reservation']):
        additional_info.append(f"예약: {row['reservation']}")
    
    details = row['details'] if pd.notna(row['details']) else ""
    
    combined_details = details + ('\n' if details and additional_info else '') + '\n'.join(additional_info)
    return combined_details

tourapi['details'] = tourapi.apply(combine_columns, axis=1)

tourapi = tourapi.drop(columns=['children_playroom', 'main_menu', 'menus', 'reservation'])


In [30]:
tourapi['source'] = 'tourAPI'
tourapi['category'] = 'restaurant'

In [31]:
tourapi = tourapi[['source', 'category', 'place_name', 'address', 'latitude', 'longitude', 'available_time', 'available_time_v2', 'parking', 'description', 'details']]

In [32]:
tourapi.head(2)

,source,category,place_name,address,latitude,longitude,available_time,available_time_v2,parking,description,details
0,tourAPI,restaurant,파크 하얏트 서울 코너스톤,서울특별시 강남구 테헤란로 6062층,37.508715,127.064265,"06:30~22:00<br>(라스트오더-세트메뉴 21:00 / 단품메뉴, 음료 21...","{'hours': {'Monday': [{'start': '06:30', 'end'...",주차 가능,"파크 하얏트 서울의 코너스톤(Cornerstone, Cucina Italiana)은...","부대시설:남녀 개별 화장실 / 업소내 위치\n부대시설:별실 4개, 바\n\n어린이놀..."
1,tourAPI,restaurant,장마담집,경기도 성남시 수정구 수정로 460-3(단대동),37.461603,127.155825,10:00~21:00 (라스트오더 20:00),"{'hours': {'Monday': [], 'Tuesday': [{'start':...",주차 가능,장마담집은 성남 남한산성 입구에 위치한 곳으로 3대째 운영중인 전통있는 백숙 전문 ...,화장실:남녀개별 / 업소 내부 위치\n\n어린이놀이터: 0.0\n메인메뉴: 닭백숙\...
